In [2]:
import psycopg2
import pandas as pd
import json 

In [1]:
def connection():
    with open("C:/Users/SEBASTIAN/ETL/dbconfig.json") as f:
        db_file=json.load(f)
    try:
        conn = psycopg2.connect(
            host = 'localhost',
            user = db_file["user"],
            password = db_file["password"],
            database = db_file["database"]
        )
        print('¡Conexion exitosa!!')
        return conn
        
    except Exception as ex:
        print(ex)
        

In [23]:
def createtable():
    conn = connection()
    cursor = conn.cursor()
    sql = '''
                 CREATE TABLE candidates
                (first_name varchar, 
                 last_name varchar,
                 email varchar,
                 application_date date,
                 country varchar,
                 YOE INT,
                 seniority varchar,
                 technology varchar,
                 code_challenge_score INT,
                 technical_interview_score INT
                );
                 '''   
    cursor.execute(sql)
    conn.commit()
    print('¡Tabla creada!')
createtable()

¡Conexion exitosa!!
¡Tabla creada!


In [24]:
def copydata():
    conn = connection()
    cursor = conn.cursor()

    sql = '''
            COPY candidates(first_name, last_name, email, application_date, country, YOE, seniority, technology, code_challenge_score, technical_interview_score)
            FROM 'C:/data/candidates.csv' DELIMITER ';' CSV HEADER;
          '''
    cursor.execute(sql)
    conn.commit()
    print('¡Registros ingresados!')
copydata()

¡Conexion exitosa!!
¡Registros ingresados!


In [34]:
def update_table():
    conn = connection()
    cursor = conn.cursor()
    update = """ALTER TABLE candidates ADD COLUMN hired boolean"""
    print("tabla actualizada")
    cursor.execute(update)
    conn.commit()
update_table()

¡Conexion exitosa!!
tabla actualizada


In [36]:
def update_hired():
    conn = connection()
    cursor = conn.cursor()
    update = """UPDATE candidates
            SET hired = (code_challenge_score >= 7 AND technical_interview_score >= 7)"""
    cursor.execute(update)
    print("Columna hired actualizada")
    conn.commit()
update_hired()

¡Conexion exitosa!!
Columna hired actualizada


In [6]:
def query1():
    conn = connection()
    cursor = conn.cursor()
    technology=[]
    Q1 = """
    SELECT technology, 
    COUNT(*) AS conteo FROM candidates WHERE hired = true GROUP BY technology  
    """
    cursor.execute(Q1)
    results=cursor.fetchall()
    for row in results:
        technology.append(row)
    df = pd.DataFrame(technology)
    df.columns=['Technology','Count']
    print(df)
    conn.commit()
query1()

¡Conexion exitosa!!
                                 Technology  Count
0                 Development - CMS Backend    284
1                                Salesforce    256
2                             Data Engineer    255
3                             QA Automation    243
4                                     Sales    239
5                                    DevOps    495
6                                 QA Manual    259
7                     System Administration    293
8                   Development - FullStack    254
9                   Database Administration    282
10                    Business Intelligence    254
11               Development - CMS Frontend    251
12                         Game Development    519
13                      Security Compliance    250
14                           Client Success    271
15                    Development - Backend    255
16                   Development - Frontend    266
17                                 Security    266
18        S

In [12]:
def query2():
    conn = connection()
    cursor = conn.cursor()
    year=[]
    Q2 = """
    SELECT EXTRACT(year FROM application_date) as year, 
    COUNT(*) AS conteo FROM candidates WHERE hired = true GROUP BY year; 
    """

    cursor.execute(Q2)
    results=cursor.fetchall()
    for row in results:
        year.append(row)
    df = pd.DataFrame(year)
    df.columns=['Year','Count']
    print(df)
    conn.commit()
query2()

¡Conexion exitosa!!
   Year  Count
0  2021   1485
1  2020   1485
2  2022    795
3  2018   1409
4  2019   1524


In [13]:
def query3():
    conn = connection()
    cursor = conn.cursor()
    seniority = []
    Q3 = """
    SELECT seniority, 
    COUNT(*) AS conteo FROM candidates WHERE hired = true GROUP BY seniority 
    """

    cursor.execute(Q3)
    results=cursor.fetchall()
    for row in results:
        seniority.append(row)
    df = pd.DataFrame(seniority)
    df.columns=['Seniority','Count']
    print(df)
    conn.commit()
query3()

¡Conexion exitosa!!
   Seniority  Count
0    Trainee    973
1     Senior    939
2  Architect    971
3     Intern    985
4       Lead    929
5  Mid-Level    924
6     Junior    977


In [16]:
def query4():
    conn = connection()
    cursor = conn.cursor()
    country = []
    Q4 = """
    SELECT EXTRACT(year FROM application_date) as year, country, 
    COUNT(*) AS conteo FROM candidates WHERE hired = true AND country IN ('Brazil','Ecuador','United States of America','Colombia') GROUP BY country, year
    """

    cursor.execute(Q4)
    results=cursor.fetchall()
    for row in results:
        country.append(row)
    df = pd.DataFrame(country)
    df.columns=['Year','Country','Count']
    print(df)
    conn.commit()
query4()

¡Conexion exitosa!!
    Year                   Country  Count
0   2018                    Brazil      9
1   2019                    Brazil      7
2   2020                    Brazil      6
3   2021                    Brazil      7
4   2022                    Brazil      4
5   2018                  Colombia      7
6   2019                  Colombia      8
7   2020                  Colombia      8
8   2021                  Colombia      1
9   2022                  Colombia      1
10  2018                   Ecuador      1
11  2019                   Ecuador      3
12  2020                   Ecuador      8
13  2021                   Ecuador      5
14  2022                   Ecuador      3
15  2018  United States of America      5
16  2019  United States of America      3
17  2020  United States of America      4
18  2021  United States of America      8
19  2022  United States of America      5
